In [ ]:
import google.auth
from google.cloud import bigquery
import pandas_gbq
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
from py2neo import Graph
import os
from ast import literal_eval
import json
from py2neo import Graph
from collections import Counter
import nltk
from nltk.collocations import *
import string
from scipy.stats import entropy
from nltk.tokenize import MWETokenizer

In [ ]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
PROCESSED_DATA = os.getenv('DIR_DATA_PROCESSED')

In [ ]:
# Connect to KG
graph = Graph(host='knowledge-graph.integration.govuk.digital',auth=(os.getenv('NEO_USER'),os.getenv('NEO_PASSWORD')), secure=True)

In [ ]:
# Connect to BQ
# Need to have active environment variable called GOOGLE_APPLICATION_CREDENTIALS pointing to json file with
# bigquery credentials
def create_big_query_client():
    credentials, project_id = google.auth.default()
    return bigquery.Client(
      credentials=credentials,
      project=project_id)

client = create_big_query_client()

In [ ]:
# Gets every page view prior to a search in a session. Multiple search queries per session are grouped together in a list
# Over a couple days its a few gigs, but the more data the better obvs
# e.g see below
"""
session_id | viewed_page | search_terms 
    123    |    /mot     |  mot, mot check
    123    | /check-mot  |  mot, mot check 
"""

query = """
SELECT
    country,
    region,
    metro,
    action.session_id,
    viewedpages.pageTitle,
    viewedpages.pagePath,
    first_search_timestamp,
    search_terms,
    pageview_timestamp,
    ROW_NUMBER() OVER (PARTITION BY action.session_id ORDER BY pageview_timestamp DESC) as hit_n
    FROM (
      SELECT
          geoNetwork.country,
          geoNetwork.region,
          geoNetwork.metro,
          CONCAT(CAST(fullVisitorId AS STRING), CAST(visitId AS STRING)) AS session_id,
          string_agg(LOWER(hits.page.searchKeyword)) as search_terms,
          MIN(TIMESTAMP_SECONDS(visitStartTime+CAST(hits.time/1000 AS INT64))) as first_search_timestamp
          FROM
          `govuk-bigquery-analytics.87773428.ga_sessions_*`,
          UNNEST(hits) AS hits
          WHERE
          _table_suffix BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY))
              AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
          AND hits.page.searchKeyword IS NOT NULL
          GROUP BY session_id, geoNetwork.region,geoNetwork.metro, geoNetwork.country
      ) AS action
LEFT JOIN (
    SELECT
        CONCAT(CAST(fullVisitorId AS STRING), CAST(visitId AS STRING)) AS session_id,
        hits.page.pageTitle as pageTitle,
        hits.page.pagePath as pagePath,
        TIMESTAMP_SECONDS(visitStartTime+CAST(hits.time/1000 AS INT64)) as pageview_timestamp,
        FROM
        `govuk-bigquery-analytics.87773428.ga_sessions_*` 
        CROSS JOIN UNNEST(hits) AS hits
        WHERE _table_suffix BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY))
          AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) 
        AND hits.type = "PAGE"
      ) as viewedpages
ON viewedpages.session_id = action.session_id
WHERE pageTitle is not null
AND pageview_timestamp < first_search_timestamp
"""

In [ ]:
#queries_df = pandas_gbq.read_gbq(query)
#queries_df.to_csv(PROCESSED_DATA+'/pagehistoryqueries.csv')
queries_df = pd.read_csv(PROCESSED_DATA+'/pagehistoryqueries.csv')


In [ ]:
# Query + session data
queries_df.head(10)

In [ ]:
# Taxons for every bit of content 
#taxons_df =graph.run("MATCH (c:Cid)-[r:IS_TAGGED_TO]->(t:Taxon)RETURN c.name as pagePath,t.name as taxon").to_data_frame()
#taxons_df.to_csv(PROCESSED_DATA+'/taxons.csv')
taxons_df =pd.read_csv(PROCESSED_DATA+'/taxons.csv')


In [ ]:
# inner join taxons to session/query data 
# We'll end up with multiple taxons per query 
# ie each query has potentially many pages viewed prior to the query being made AND pages can have multiple taxons

query_taxons = queries_df.merge(taxons_df,on='pagePath')
# Split queries up (they're stupidily aggregated in the sql)
query_taxons['query']=query_taxons['search_terms'].map(lambda x: x.split(','))
query_taxons = query_taxons.explode('query')
# Average number of taxons visited per query (around 7)
np.mean(query_taxons.groupby('session_id').size())

In [ ]:
# Text preprocessing
translator = str.maketrans('', '', string.punctuation)
# Some multiword tokens
tokenizer = MWETokenizer([('log', 'in'), ('sign', 'in'), ('sign', 'up')])
# Tokenise query, explode, and regroup by taxon 
# End up with a list of tokens per taxon
query_taxons['tokens'] = query_taxons['query'].map(lambda x: tokenizer.tokenize(x.translate(translator).split()))
query_taxons = query_taxons.explode('tokens')
query_taxons = query_taxons[['taxon','tokens']]
query_taxons = query_taxons.dropna()
# Group query tokens by taxon so we have a list of tokens per taxon
query_taxons = query_taxons.groupby('taxon').aggregate(lambda x: list(x)).reset_index()

In [ ]:
# All the maintstream content text,titles from gov uk (assumed targets for users)
#title_df =graph.run("match (m:Mainstream) return m.name as name, m.title as title, m.text as text").to_data_frame()
#title_df.to_csv(PROCESSED_DATA+'/mainstreamcontent.csv')
title_df = pd.read_csv(PROCESSED_DATA+'/mainstreamcontent.csv')
title_df = title_df.dropna()

In [ ]:
#We want to weight titles a bit highly, so err just multiply by 3
title_df['body_tokens'] = title_df['text'] + ' ' + (title_df['title'] + ' ' * 3)
title_df['body_tokens'] = title_df['body_tokens'].map(lambda x: tokenizer.tokenize(x.translate(translator).split()))
title_df['body_tokens'] = title_df['body_tokens'].map(lambda x: [token.lower() for token in x])
title_df = title_df.dropna()



In [ ]:
title_df.head(1)

In [ ]:
# Intersection of words in queries + content
service_terms= set(title_df['body_tokens'].explode('body_tokens'))
# Lose some stop words
intersection = service_terms.difference(set(['i','of','the','a','and','to','you','if','that','then','in','on']))

In [ ]:
title_df['intersect_tokens'] = title_df['body_tokens'].map(lambda x: [token for token in x if token in intersection])
title_df['body_token_count'] = title_df['intersect_tokens'].map(lambda x: Counter(x))

In [ ]:
# bag of words for mainstream content
service_bow = pd.DataFrame.from_records(title_df['body_token_count'],index=title_df['name'])
service_bow = service_bow.replace(np.nan,0)
#add one smoothing
service_bow = service_bow +1
service_dis = service_bow.divide(service_bow.sum(1),0)
service_dis.shape

In [ ]:
query_taxons = query_taxons[['taxon','tokens']]
query_taxons['valid_tokens'] = query_taxons['tokens'].map(lambda x: [token for token in x if token in intersection])
query_taxons['valid_token_count'] = query_taxons['valid_tokens'].map(lambda x:Counter(x))

In [ ]:
# Bag of words for taxon queries
query_bow = pd.DataFrame.from_records(query_taxons['valid_token_count'],index=query_taxons['taxon'],columns=service_dis.columns)
# Don't smooth unless you have lots of data
query_bow = query_bow.replace(np.nan,0)

query_dis = query_bow.divide(query_bow.sum(1),0)
query_dis.shape


In [ ]:
assert(query_dis.shape[1]==service_dis.shape[1]==len(intersection))


In [ ]:
# Find min KL divergence content to taxon query terms
# https://en.wikipedia.org/wiki/Information_projection
# Add one smoothing, which is p, which is q, all make quite a big difference in performance
# p is our normalised bag of words from each taxon query
# q is every bit of mainstream content normalised bag of words
# e.g try 'Blue badges', 'Afghanistan' etc
TAXON = "Stopping or selling your business"
idx = np.argsort(entropy(np.broadcast_to(query_dis.loc[TAXON].to_numpy(),(service_dis.shape[0],service_dis.shape[1])),service_dis,axis=1))[0:15]
service_bow.iloc[idx].index

In [ ]:
# Try the same with transition checker data
# Same query as above but only looking for transition checker pages completed prior to searching
acc_query="""SELECT
  action.session_id,
  viewedpages.pageTitle,
  viewedpages.pagePath,
  first_search_timestamp,
  search_terms,
  pageview_timestamp,
  ROW_NUMBER() OVER (PARTITION BY action.session_id ORDER BY pageview_timestamp DESC) as hit_n
  FROM (
   SELECT
   geoNetwork.country,
   geoNetwork.region,
   geoNetwork.metro,
  CONCAT(CAST(fullVisitorId AS STRING), CAST(visitId AS STRING)) AS session_id,
  string_agg(LOWER(hits.page.searchKeyword)) as search_terms,
  MIN(TIMESTAMP_SECONDS(visitStartTime+CAST(hits.time/1000 AS INT64))) as first_search_timestamp

FROM
  `govuk-bigquery-analytics.87773428.ga_sessions_*`,
  UNNEST(hits) AS hits
WHERE
  _table_suffix BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY))
                AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
  AND hits.page.searchKeyword IS NOT NULL
   group by session_id, geoNetwork.region,geoNetwork.metro, geoNetwork.country
  ) AS action
  LEFT JOIN 
  (SELECT 
  CONCAT(CAST(fullVisitorId AS STRING), CAST(visitId AS STRING)) AS session_id,
  hits.page.pageTitle as pageTitle,
  hits.page.pagePath as pagePath,
  ROW_NUMBER() OVER (PARTITION BY CONCAT(CAST(fullVisitorId AS STRING), CAST(visitId AS STRING)) ORDER BY hits.time DESC) as last_question,
  TIMESTAMP_SECONDS(visitStartTime+CAST(hits.time/1000 AS INT64)) as pageview_timestamp,
  FROM
  `govuk-bigquery-analytics.87773428.ga_sessions_*` 
  CROSS JOIN UNNEST(hits) AS hits
  WHERE 
      _table_suffix BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 3 DAY))
      AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) 
      AND hits.type = "PAGE"
      AND hits.page.pagePath LIKE '/transition-check/questions?%'
  ) as viewedpages
  ON viewedpages.session_id = action.session_id
  where pageTitle is not null
  AND last_question=1
  AND pageview_timestamp < first_search_timestamp"""

In [ ]:
#acc_df = pandas_gbq.read_gbq(acc_query)
#acc_df.to_csv(PROCESSED_DATA+'/accounts.csv')
#Clean data stored in urls (annoying)
#acc_df['responses']= acc_df['pagePath'].map(lambda x:x.split('c[]'))
#acc_df = acc_df.explode('responses')
#acc_df['clean'] = acc_df['responses'].map(lambda x: x.split('&')[0])
#acc_df = acc_df[acc_df['clean']!='/transition-check/questions?page=1']
#acc_df = acc_df[acc_df['clean']!='/transition-check/questions?']
#acc_df['clean'] = acc_df['clean'].map(lambda x: x.replace('=',''))

acc_df = pd.read_csv(PROCESSED_DATA+'/accounts.csv')

In [ ]:
acc_df

In [ ]:
#Check whats worth conditioning on
acc_df.groupby('clean').size().reset_index(name='count').sort_values('count',ascending=False)


In [ ]:
# Tokenise search terms
acc_df['search_terms'] = acc_df['search_terms'].map(lambda x: x.split(','))
acc_df = acc_df.explode('search_terms')
acc_df['tokens'] = acc_df['search_terms'].map(lambda x: tokenizer.tokenize(x.translate(translator).split()))
acc_df['valid_tokens'] = acc_df['tokens'].map(lambda x: [token for token in x if token in service_terms])


In [ ]:
acc_df = acc_df.explode('valid_tokens')
acc_df = acc_df.dropna()

In [ ]:
acc_df

In [ ]:
# Let's try conditioning search terms on nationality
nationality = acc_df[acc_df['clean'].isin(['nationality-eu','nationality-uk','nationality-row'])]
nationality = nationality[['clean','valid_tokens']].groupby('clean').aggregate(lambda x: list(x))
nationality['valid_token_count'] = nationality['valid_tokens'].map(lambda x:Counter(x))
nationality = nationality.reset_index()

In [ ]:
nationality

In [ ]:
# Bag of words for nationality search terms
nat_bow = pd.DataFrame.from_records(nationality['valid_token_count'],columns=service_dis.columns,index=nationality['clean'])                                  
nat_bow = nat_bow.replace(np.nan,0) + 1
nat_bow = nat_bow.divide(nat_bow.sum(1),0)
nat_bow

In [ ]:
idx = np.argsort(entropy(np.broadcast_to(nat_bow.loc['nationality-uk'].to_numpy(),(service_dis.shape[0],service_dis.shape[1])),service_dis,axis=1))[0:20]
service_bow.iloc[idx].index

In [ ]:
idx = np.argsort(entropy(np.broadcast_to(nat_bow.loc['nationality-eu'].to_numpy(),(service_dis.shape[0],service_dis.shape[1])),service_dis,axis=1))[0:10]
service_bow.iloc[idx].index

In [ ]:
idx = np.argsort(entropy(np.broadcast_to(nat_bow.loc['nationality-row'].to_numpy(),(service_dis.shape[0],service_dis.shape[1])),service_dis,axis=1))[0:10]
service_bow.iloc[idx].index

In [ ]:
taxons_df

In [ ]:
# Let's look for clusters within a taxon-query distribution